In [1]:
import tensorflow as tf
from keras_cv_attention_models import model_surgery

In [2]:
model = tf.keras.models.load_model('ridnet.h5', compile=False)

In [3]:
new_width = 1280
new_height = 720

In [4]:
model2 = model_surgery.change_model_input_shape(model, (new_height, new_width, 3))

>>>> Changed model input shape from (None, 40, 40, 3) to (None, 720, 1280, 3)


In [5]:
model2.set_weights(model.get_weights())

In [6]:
concrete = tf.function(lambda inputs: model2(inputs))
concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *t.shape[1:]]) for t in model2.inputs])

In [7]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()
with open(f'ridnet_{new_width}_{new_height}.tflite', 'wb') as fout:
    fout.write(tflite_model)